In [1]:
# !rm -rf build



In [61]:
# !python setup.py install

In [60]:
# pip install tensorflow

In [59]:
# pip install tensorflow_privacy

In [1]:
!pip list | grep tensorflow

tensorflow                    2.14.0
tensorflow-datasets           4.9.3
tensorflow-estimator          2.14.0
tensorflow-io-gcs-filesystem  0.34.0
tensorflow-macos              2.14.0
tensorflow-privacy            0.8.11
tensorflow-probability        0.20.1


In [2]:
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()

import numpy as np

tf.get_logger().setLevel('ERROR')


Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy


In [53]:
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer


In [58]:
# pip install 'dvc-gdrive'

In [62]:
# !dvc init

In [7]:
!git checkout 34bd5a261208081c21f975c08ea19a75afc53448 /Users/kajalshukla/Desktop/MLOPs/Assignments/Assignment-1/adsp_ml_ops/data/athletes.csv.dvc  # version-2

Updated 0 paths from 75509c6


In [8]:
!dvc pull /Users/kajalshukla/Desktop/MLOPs/Assignments/Assignment-1/adsp_ml_ops/data/athletes.csv.dvc

Fetching
!
  0% Checking cache in '/Users/kajalshukla/Desktop/MLOPs/Assignments/Assignment-
Fetching                                                                        
Building workspace index                             |2.00 [00:00, 1.17kentry/s]
Comparing indexes                                    |3.00 [00:00, 2.26kentry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
Everything is up to date.


In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd

In [11]:
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()

import numpy as np

tf.get_logger().setLevel('ERROR')

In [39]:
# Load the data
data = pd.read_csv('/Users/kajalshukla/Desktop/MLOPs/Assignments/Assignment-1/adsp_ml_ops/data/athletes.csv')

In [40]:
data.head(4)

,age,height,weight,candj,snatch,deadlift,backsq,total_lift,region_Asia,region_Australia,...,howlong_Less than 6 months|6-12 months|1-2 years|2-4 years|4+ years|Decline to answer|,howlong_Less than 6 months|Decline to answer|,bmi,age_group_Young Adult,age_group_Adult,age_group_Senior,weight_group_Normal weight,weight_group_Overweight,weight_group_Obese,weight_group_Extremely Obese
0,30,71,200,235,175,385,315,1110,0,0,...,0,0,396,1,0,0,0,1,0,0
1,28,70,176,187,134,335,254,910,0,0,...,0,0,359,1,0,0,0,1,0,0
2,35,68,225,285,205,440,405,1335,0,0,...,0,0,486,0,1,0,0,0,1,0
3,36,71,199,267,212,485,390,1354,0,0,...,0,0,394,0,1,0,0,1,0,0


### Load and Split X_train and y_train

In [41]:
#Split data for data_v1
X  = data.drop('total_lift', axis=1)
y  = data['total_lift']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

# Part - 13

### Set Differential Privacy Parameters

In [42]:
# Hyperparameters

learning_rate = 0.001
epochs = 10
batch_size = 250
n_features = X_train.shape[1]

In [43]:
l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 1

### Create a Model

In [44]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_dim=X_train.shape[1])
])

### Use Differential Privacy Optimizer

In [54]:
optimizer = DPGradientDescentGaussianOptimizer(
    l2_norm_clip = l2_norm_clip,
    noise_multiplier = noise_multiplier,
    num_microbatches = num_microbatches,
    learning_rate = learning_rate)

In [55]:
from keras.models import Sequential
from keras.layers import Dense

input_shape = X_train.shape[1]  # Number of features

model = Sequential()
model.add(Dense(128, input_shape=(input_shape,), activation='relu'))
model.add(Dense(1, activation='linear'))  # Assuming it's a regression problem

### Compile and Train the Model:

In [56]:
model.compile(optimizer=optimizer, loss='mean_squared_error')  # Assuming a regression problem


In [57]:
epochs = 10  # or whatever value you've chosen
batch_size = 1  # Using a batch size of 1 for simplicity, but you can adjust

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)


Train on 24023 samples
Epoch 1/10
  171/24023 [..............................] - ETA: 14s - loss: 837484.6282  

2023-10-17 22:07:17.424622: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-10-17 22:07:17.436681: W tensorflow/c/c_api.cc:305] Operation '{name:'TFOptimizer/iterations/Assign' id:68 op device:{requested: '', assigned: ''} def:{{{node TFOptimizer/iterations/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_INT64, validate_shape=false](TFOptimizer/iterations, TFOptimizer/iterations/Initializer/initial_value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


24023/24023 [==============================] - 13s 535us/sample - loss: 12692.0402
Epoch 2/10
24023/24023 [==============================] - 13s 531us/sample - loss: 2196.6083
Epoch 3/10
24023/24023 [==============================] - 13s 541us/sample - loss: 2102.5663
Epoch 4/10
24023/24023 [==============================] - 13s 540us/sample - loss: 5295.8608
Epoch 5/10
24023/24023 [==============================] - 13s 559us/sample - loss: 3673.3687
Epoch 6/10
24023/24023 [==============================] - 13s 540us/sample - loss: 5116.9607
Epoch 7/10
24023/24023 [==============================] - 13s 547us/sample - loss: 7529.1802
Epoch 8/10
24023/24023 [==============================] - 13s 530us/sample - loss: 7500.0398
Epoch 9/10
24023/24023 [==============================] - 12s 519us/sample - loss: 9163.5780
Epoch 10/10
24023/24023 [==============================] - 12s 512us/sample - loss: 19303.8495


### Evaluate and Get Metrics:

In [149]:
metrics = model.evaluate(X_test, y_test)
print("Test Loss:", metrics)


Test Loss: 18871.213699711745


# Part - 14

In [67]:
n = 10000  # Number of training examples.
batch_size = 256
noise_multiplier = 1.1
epochs = 5
delta = 1e-5

In [131]:
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

In [132]:
dir(compute_dp_sgd_privacy)


['_BATCH_SIZE',
 '_DELTA',
 '_MAX_EXAMPLES_PER_USER',
 '_NOISE_MULTIPLIER',
 '_NUM_EPOCHS',
 '_NUM_EXAMPLES',
 '_USED_MICROBATCHING',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'app',
 'compute_dp_sgd_privacy_statement',
 'flags',
 'main']

In [144]:
from tensorflow_privacy.privacy.analysis.compute_dp_sgd_privacy_lib import compute_dp_sgd_privacy_statement


In [145]:
def compute_dp_sgd_privacy(N, batch_size, epochs, noise_multiplier, delta=1e-6, used_microbatching=True, max_examples_per_user=None):
    statement = compute_dp_sgd_privacy_statement(
        N, batch_size, epochs, noise_multiplier, delta, used_microbatching, max_examples_per_user
    )
    return statement

# Example usage
N = 60000
batch_size = 256
noise_multiplier = 1.12
epochs = 60
delta = 1e-5

In [146]:
result = compute_dp_sgd_privacy(N, batch_size, epochs, noise_multiplier, delta)
print(result)

DP-SGD performed over 60000 examples with 256 examples per iteration, noise
multiplier 1.12 for 60 epochs with microbatching, and no bound on number of
examples per user.

This privacy guarantee protects the release of all model checkpoints in addition
to the final model.

Example-level DP with add-or-remove-one adjacency at delta = 1e-05 computed with
RDP accounting:
    Epsilon with each example occurring once per epoch:       160.398
    Epsilon assuming Poisson sampling (*):                     15.344

No user-level privacy guarantee is possible without a bound on the number of
examples per user.

(*) Poisson sampling is not usually done in training pipelines, but assuming
that the data was randomly shuffled, it is believed the actual epsilon should be
closer to this value than the conservative assumption of an arbitrary data
order.



# 15. Compare and comment on the accuracy/metrics of the non-DP and DP models using dataset v2.


Non-DP Model:
MSE: 6.188364302790066e-17 (almost zero)
R2 Score: 1.0 (perfect score)

DP Model:
Epsilon Value: Based on your previous input, the ε (epsilon) value is 160.398 for each example occurring once per epoch and 15.344 assuming Poisson sampling. A smaller epsilon value means stronger privacy but typically at the cost of model accuracy.
Test Loss (presumably MSE): 18871.213699711745

The non-DP model has significantly better accuracy compared to the DP model for dataset v2. The high test loss in the DP model indicates that the added noise, necessary to ensure differential privacy, has affected the model's predictive capabilities. This showcases the trade-off between ensuring user privacy and model accuracy. The specific ε values provide insight into the privacy guarantee of the DP model, but they also indicate a potential reason for the reduced accuracy compared to the non-DP model.

In summary, while the non-DP model achieves near-perfect accuracy, the DP model, due to its differential privacy constraints, has reduced accuracy but offers a privacy guarantee, as indicated by its ε values. The choice between the two would depend on the specific requirements of your application – whether you prioritize accuracy or user privacy.

In [150]:
!git status

On branch monika_v2
Your branch is up to date with 'origin/monika_v2'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../.DS_Store
	deleted:    ../data_privacy.ipynb
	modified:   ../eda.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	./

no changes added to commit (use "git add" and/or "git commit -a")


In [151]:
!git add .

In [152]:
!git status

On branch monika_v2
Your branch is up to date with 'origin/monika_v2'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   .bazelrc
	new file:   CONTRIBUTING.md
	new file:   LICENSE
	new file:   README.md
	new file:   WORKSPACE
	new file:   build/lib/tensorflow_privacy/__init__.py
	new file:   build/lib/tensorflow_privacy/privacy/__init__.py
	new file:   build/lib/tensorflow_privacy/privacy/analysis/__init__.py
	new file:   build/lib/tensorflow_privacy/privacy/analysis/compute_dp_sgd_privacy.py
	new file:   build/lib/tensorflow_privacy/privacy/analysis/compute_dp_sgd_privacy_lib.py
	new file:   build/lib/tensorflow_privacy/privacy/analysis/compute_dp_sgd_privacy_test.py
	new file:   build/lib/tensorflow_privacy/privacy/analysis/compute_noise_from_budget.py
	new file:   build/lib/tensorflow_privacy/privacy/analysis/compute_noise_from_budget_lib.py
	new file:   build/lib/tensorflow_privacy/privacy/analysis/compute_noise_from_budget_test.py
	new file

In [153]:
!git commit -am "update data privacy file"

[monika_v2 98f5c0f] update data privacy file
 523 files changed, 92596 insertions(+), 2149 deletions(-)
 delete mode 100644 data_privacy.ipynb
 create mode 100644 privacy-0.8.11/.bazelrc
 create mode 100644 privacy-0.8.11/CONTRIBUTING.md
 create mode 100644 privacy-0.8.11/LICENSE
 create mode 100644 privacy-0.8.11/README.md
 create mode 100644 privacy-0.8.11/WORKSPACE
 create mode 100644 privacy-0.8.11/build/lib/tensorflow_privacy/__init__.py
 create mode 100644 privacy-0.8.11/build/lib/tensorflow_privacy/privacy/__init__.py
 create mode 100644 privacy-0.8.11/build/lib/tensorflow_privacy/privacy/analysis/__init__.py
 create mode 100644 privacy-0.8.11/build/lib/tensorflow_privacy/privacy/analysis/compute_dp_sgd_privacy.py
 create mode 100644 privacy-0.8.11/build/lib/tensorflow_privacy/privacy/analysis/compute_dp_sgd_privacy_lib.py
 create mode 100644 privacy-0.8.11/build/lib/tensorflow_privacy/privacy/analysis/compute_dp_sgd_privacy_test.py
 create mode 100644 privacy-0.8.11/build/lib/t

In [154]:
!git push

Enumerating objects: 417, done.
Counting objects: 100% (417/417), done.
Delta compression using up to 8 threads
Compressing objects: 100% (405/405), done.
Writing objects: 100% (414/414), 2.16 MiB | 5.24 MiB/s, done.
Total 414 (delta 100), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (100/100), completed with 2 local objects.
To https://github.com/mariafshan/adsp_ml_ops.git
   71cfb73..98f5c0f  monika_v2 -> monika_v2
